In [14]:
import carla 
import random

client = carla.Client('localhost', 2000)
client.set_timeout(100.0)

world = client.get_world()


In [15]:

# load a map
client.load_world('Town10HD')


In [16]:
# adding vehicles and NPCs
# spawn vehicles
# get all the blueprints of the vehicles
vehicle_blueprints = world.get_blueprint_library().filter('vehicle.*')
# each map provides predefined spawn points
spawn_points = world.get_map().get_spawn_points()
# spawn 50 vehicles randomly distributed around the map
# for each spawn point, we randomly choose a blueprint and spawn a vehicle
for i in range(10):
    spawn_point = random.choice(spawn_points)
    blueprint = random.choice(vehicle_blueprints)
    world.try_spawn_actor(blueprint, spawn_point)


In [23]:

# now we should add ego vehicle with a given name
# if raise RuntimeError, try second time 
ego_bp = world.get_blueprint_library().find('vehicle.tesla.model3')
ego_bp.set_attribute('role_name', 'hero')
ego_vehicle = world.try_spawn_actor(ego_bp, random.choice(spawn_points))



In [29]:
# try to move spector to ego vehicle
spector = world.get_spectator()
ego_transform = ego_vehicle.get_transform()

# set the position of the spector
spector.set_transform(carla.Transform(ego_transform.location + carla.Location(z=5),
                                      ego_transform.rotation))


Add sensors to vehicle 


In [30]:
# create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))
# create the camera throught a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
# spawn the camera and attach it to the ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)


Once we have spawned the camera, we need to set it recording through the listen() method. The listen method takes as argument a callback that defines what to do with the data. You can either stream it to another program or save it to disk.

We will use a lambda function as a callback to save the data to disk:

In [ ]:
# start camera with PyGame callback()
#camera.listen(lambda image: image.save_to_disk('output/%06d.png' % image.frame))

## Animate vehicles

In [31]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)
